In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Hyper-parameters
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data',
                                          train=False,
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [5]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [6]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [7]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}')

Epoch [1/5], Step [100/600], Loss: 0.3335
Epoch [1/5], Step [200/600], Loss: 0.3213
Epoch [1/5], Step [300/600], Loss: 0.3330
Epoch [1/5], Step [400/600], Loss: 0.1248
Epoch [1/5], Step [500/600], Loss: 0.1065
Epoch [1/5], Step [600/600], Loss: 0.1432
Epoch [2/5], Step [100/600], Loss: 0.2916
Epoch [2/5], Step [200/600], Loss: 0.0629
Epoch [2/5], Step [300/600], Loss: 0.1226
Epoch [2/5], Step [400/600], Loss: 0.1699
Epoch [2/5], Step [500/600], Loss: 0.1762
Epoch [2/5], Step [600/600], Loss: 0.0535
Epoch [3/5], Step [100/600], Loss: 0.0300
Epoch [3/5], Step [200/600], Loss: 0.1948
Epoch [3/5], Step [300/600], Loss: 0.0733
Epoch [3/5], Step [400/600], Loss: 0.0452
Epoch [3/5], Step [500/600], Loss: 0.0840
Epoch [3/5], Step [600/600], Loss: 0.0348
Epoch [4/5], Step [100/600], Loss: 0.0406
Epoch [4/5], Step [200/600], Loss: 0.0278
Epoch [4/5], Step [300/600], Loss: 0.0330
Epoch [4/5], Step [400/600], Loss: 0.0593
Epoch [4/5], Step [500/600], Loss: 0.0680
Epoch [4/5], Step [600/600], Loss:

In [9]:
# Test the model
# In the test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')

Accuracy of the network on the 10000 test images: 97.68 %


In [10]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')